In [1]:
import shutup
shutup.please()

import pandas as pd
import numpy as np
import os, os.path, pickle, sys
from datetime import datetime, date

In [2]:
wheat_database = "/Users/hn/Documents/01_research_data/Other_people/Ehsan/wheat/"
data_dir = wheat_database + "data/"
separate_varieties_dir = data_dir + "varieties/"

reOrganized_dir = data_dir + "reOrganized/"
os.makedirs(reOrganized_dir, exist_ok=True)

In [3]:
# This is not what we want.
# We want full data series.
# filename = (reOrganized_dir + "all_stages_df22805_varietyAvgd.sav")
# all_stages_df22805_varietyAvgd = pd.read_pickle(filename)
# all_stages_df22805_varietyAvgd.keys()

# all_stages_df22805_varietyAvgd = all_stages_df22805_varietyAvgd["all_stages_data"]
# all_stages_df22805_varietyAvgd.head(2)

In [4]:
filename = (reOrganized_dir + "variables_dict.sav")
variables_dict = pd.read_pickle(filename)
variables_dict.keys()

variables_dict = variables_dict["variables_abb_dict"]
variables_dict

{'fdd': 'freezing_dd',
 'srad': 'shortwave_rad_Wm2',
 'prdtr': 'precip_div_dtr',
 'dtr': 'diurnal_temp',
 'dgdd': 'diurnal_gdd',
 'ravg': 'relative_humidity_avg',
 'hdd': 'high_dd',
 'vs': 'wind_speed_ms',
 'vpd': 'vpd_kPa'}

In [5]:
filename = (reOrganized_dir + "average_and_seperate_varieties_weekly.sav")
average_and_seperate_weekly = pd.read_pickle(filename)
average_and_seperate_weekly.keys()

dict_keys(['averaged_varieties_weekly', 'separate_varieties_weekly', 'separate_varieties_annual', 'averaged_varieties_annual', 'separate_varieties_4season', 'averaged_varieties_4season', 'dates', 'source_code', 'Author', 'Date'])

In [7]:
separate_varieties_weekly = average_and_seperate_weekly["separate_varieties_weekly"]
separate_varieties_weekly.head(2)

,location,year,variety,yield,1_tavg,2_tavg,3_tavg,4_tavg,5_tavg,6_tavg,...,16_vs,17_vs,18_vs,19_vs,20_vs,21_vs,22_vs,23_vs,24_vs,25_vs
0,Almira,2005,Alpowa,43.9,7.725714,14.655,11.125714,14.511429,14.647143,12.345,...,3.3,2.528571,2.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Almira,2005,Alturas,37.7,7.725714,14.655,11.125714,14.511429,14.647143,12.345,...,3.3,2.528571,2.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import re

col_names = list(separate_varieties_weekly.columns)
col_names[:4]

['location', 'year', 'variety', 'yield']

In [19]:
## detect columns that start with a digit
## so we can extract them and put them in a list as time series.
pattern = r"^\d"
digital_columns = [s for s in col_names if re.match(pattern, s)]
digital_columns[:4]

['1_tavg', '2_tavg', '3_tavg', '4_tavg']

In [21]:
print (f"{len(digital_columns) = }")
print (f"{len(col_names) = }")

len(digital_columns) = 325
len(col_names) = 329


In [22]:
non_digital_columns = [s for s in col_names if not(re.match(pattern, s))]
non_digital_columns[:4]

['location', 'year', 'variety', 'yield']

In [47]:
# count number of different variables
variables_list = [s.split("_")[1] for s in digital_columns]
variables_list = list(set(variables_list))
variables_list.remove("dtr.1")
variables_list

['tavg',
 'dtr',
 'hdd',
 'vs',
 'gdd',
 'vpd',
 'ravg',
 'dgdd',
 'srad',
 'precip',
 'prdtr',
 'fdd']

In [52]:
rows = len(separate_varieties_weekly)
cols = len(non_digital_columns) + len(variables_list)
separate_vars_weekly_TS = pd.DataFrame(np.zeros((rows, cols)))
separate_vars_weekly_TS.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
columns = non_digital_columns + variables_list
separate_vars_weekly_TS.columns = columns
separate_vars_weekly_TS.head(2)

,location,year,variety,yield,tavg,dtr,hdd,vs,gdd,vpd,ravg,dgdd,srad,precip,prdtr,fdd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Populate dataframe

In [54]:
separate_vars_weekly_TS[non_digital_columns] = separate_varieties_weekly[non_digital_columns]
separate_vars_weekly_TS.head(2)

,location,year,variety,yield,tavg,dtr,hdd,vs,gdd,vpd,ravg,dgdd,srad,precip,prdtr,fdd
0,Almira,2005,Alpowa,43.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Almira,2005,Alturas,37.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for a_variable in variables_list:
    